### LLMChain 
adds some functionality around language models

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

### RouterChain

Create a chain that dynamically selects the next chain to use for a given input.

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

### Define the two LLMChains

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

llm = OpenAI()

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

### Define the RouterChain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)







> Entering new MultiPromptChain chain...


/home/nurbot/miniconda3/envs/hf/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation is the electromagnetic radiation emitted by an idealized physical body that is in thermal equilibrium with its environment. It is a kind of thermal radiation, resulting from the spontaneous emission of photons from atoms at any temperature above absolute zero. The spectrum of this radiation is determined by the temperature of the body, and is independent of the material composition of the body.


In [3]:
chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"\n\nBlack body radiation is the electromagnetic radiation emitted from an idealized physical body that is in thermal equilibrium with its environment. It is emitted by any object that has an internal energy distribution, such as a star. The spectrum of black body radiation is determined by the temperature of the body and is independent of the body's shape or composition. It is also known as thermal radiation or thermal emission."

In [4]:
print(
    chain.run(
        "What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?"
    )
)



> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3?'}
> Finished chain.


The first prime number greater than 40 such that one plus the prime number is divisible by 3 is 43. To solve this problem, we can use the fact that prime numbers are only divisible by themselves and 1, so any prime number greater than 40 that is divisible by 3 must also have a remainder of 1 when divided by 3. This means that the number must be congruent to 1 modulo 3, which in turn means that the number must be of the form 3k+1, where k is an integer. Starting with k=1, we get 3*1+1=4, which is not prime, so we try k=2: 3*2+1=7, which is prime, and one plus this number (7+1=8) is divisible by 3. Therefore, the first prime number greater than 40 such that one plus the prime number is divisible by 3 is 7.


### Sequential

series of calls to a language model.

### Transformation
An example, create a dummy transformation that takes in a super long text, filters the text to only the first 3 paragraphs, and then passes that into an LLMChain to summarize those.